# XGBoosting

Leitura dos datasets tratados

In [1]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('../datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('../datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)

## Construção e modelação

For the xgBoost model we need to encode the target feature as well.

In [2]:
order_mapping = {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very High': 4}
treino['Injeçao na rede (kWh)']= treino['Injeçao na rede (kWh)'].map(order_mapping)

In [3]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

In [4]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

xgb_estimator = xgb.XGBClassifier(random_state=2023)

params = {
    'n_estimators': [10, 50, 100, 200], 'learning_rate': [0.05, 0.1, 0.3, 0.5], 'max_depth': list(range(1,16))
}

grid = GridSearchCV(estimator=xgb_estimator,
                    param_grid=params,
                    cv=5,
                    n_jobs=5,
                    verbose=2)

# xgb_estimator.fit(X_treino, y_treino)
grid.fit(X_treino, y_treino)
grid.best_score_, grid.best_params_

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


(0.8880000000000001,
 {'learning_rate': 0.3, 'max_depth': 14, 'n_estimators': 200})

In [6]:
reverse_order_mapping = {0: 'None',1: 'Low',2: 'Medium',3: 'High',4: 'Very High'}

predictions = grid.predict(X_teste)

predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df['Result'] = df['Result'].map(reverse_order_mapping) 
df.index += 1
df.index.name = 'RowId'

df.to_csv(f'predictions/xgbPrediction.csv')